# Landscape Elevation Connectivity of a single point

Here the goal is to pick a single point, somewhere in our mesh, and calculate its Landscape Elevation Connectivity (LEC) value.

This notebook will show you:
- how to load a mesh into `gLEC`, 
- how to use some of its functions, including calculating the LEC, for a single point,

This notebook includes a lot of code for visualising the mesh, and the point on it. These bits of code are not necessary for using gLEC, so feel free to skip reading those code chunks. They will be marked as:
```python
# VISUALISATION CODE
```

In [ ]:
import meshio
import numpy as np
from multiprocessing import Pool
import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.path import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset
import matplotlib.tri as mtri

label_size = 8
matplotlib.rcParams['xtick.labelsize'] = label_size 
matplotlib.rcParams['ytick.labelsize'] = label_size

%matplotlib inline
%config InlineBackend.figure_format = 'svg' 

from gLEC.gLEC import gLEC

# Load the mesh

The first step is to load in a mesh file, using `meshio`. This notebook assumes you have run the `1-Prepare_mesh` notebook first, however, it should work OK with other meshes.

In [ ]:
infile = "output_data/AUS_LR.vtk"

mesh = meshio.read(infile)

We can visualise it with matplotlib:

In [ ]:
# VISUALISATION CODE
triang = mtri.Triangulation(mesh.points[:,0], mesh.points[:,1], mesh.cells[0].data)
fig, ax = plt.subplots()
ax.triplot(triang, zorder=1)
ax.set_aspect('equal')

# Create a gLEC object

Now that we have a mesh, let's create a `gLEC` object. Beyond just calculating the LEC of a region, the `gLEC` object can help us with a few other tasks too. 

Creating a basic `gLEC` object is done like this:

In [ ]:
lec_calculator = gLEC(mesh, max_fuel = 1000)

# Pick a starting point

In this notebooks, we want to do an LEC analysis of a single point - so let's pick a starting point!

We don't need a particular point, so we could do something like this:
```python
starting_point = 0
```
which would use the first point defined in the mesh.

However, for this example, it is useful to pick a point that is easy to see and distinquish, so we use this function to find a better one:

In [ ]:
starting_point = mesh.points.shape[0]//9 * 8

In [ ]:
print(f'Starting point index: {starting_point}')
print(f'Starting point value:\n{mesh.points[starting_point,:]}')

We can now see the point index, and also the X, Y, and Z values for it.

However, it's much easier to understand if we visualise where the point is:

In [ ]:
# VISUALISATION CODE
fig, ax = plt.subplots()
ax.triplot(triang, zorder=1)
ax.set_aspect('equal')

# Plot the starting point on as a yellow dot
ax.scatter(mesh.points[starting_point,0],
           mesh.points[starting_point,1],
           c='yellow', s =20, zorder=3)

# Find a nearby neighbour

Before we do a full LEC analysis of this single point, we can do some quick checks of our data, and use some of the functions `gLEC` can provide.

To do this, we need to find a point that is a nearby neighbour of our chosen starting point. `gLEC` has a function for doing this for us:

In [ ]:
nearby_points = lec_calculator.neighbours_func(starting_point)

In [ ]:
print(f'Nearby points indexs: {nearby_points}')
print(f'Nearby points values:\n{mesh.points[nearby_points,:]}')

This shows that our starting point has a number of neighbours which it's directly connected to.

We can visualise them, to get a better understanding of their connection

In [ ]:
# VISUALISATION CODE

# Choose the size and zoom of the inset
zoom_min = np.min(mesh.points[nearby_points,:], axis=0)
zoom_max = np.max(mesh.points[nearby_points,:], axis=0)
zoom_span = zoom_max - zoom_min
zoom_min -= zoom_span * 0.5
zoom_max += zoom_span * 0.5

total_min = np.min(mesh.points, axis=0)
total_max = np.max(mesh.points, axis=0)
total_span = total_max - total_min

# Calculate how much zoom is needed for the size of mesh
screen_percentage = 0.15
zoom_factor = min(1 / (zoom_span[:2] / (total_span[:2] * screen_percentage)))

fig, ax = plt.subplots(figsize=(10,10))

# Plot the mesh at full scale
ax.triplot(triang, zorder=1)

# Plot the starting point at full scale
ax.scatter(mesh.points[starting_point,0],
           mesh.points[starting_point,1],
           c='yellow', s =20, zorder=3)

# Plot the nearby points at full scale
ax.scatter(mesh.points[nearby_points,0],
           mesh.points[nearby_points,1],
           c='orange', s =20, zorder=2)
ax.set_aspect('equal')

# Now make an inset, and plot all the data again at that scale
axins = zoomed_inset_axes(ax, zoom_factor, loc=2)

# Plot the mesh 
axins.triplot(triang, zorder=1)

# Plot the starting point
axins.scatter(mesh.points[starting_point,0],
           mesh.points[starting_point,1],
           c='yellow', s =20, zorder=3)

# Plot the nearby points
axins.scatter(mesh.points[nearby_points,0],
           mesh.points[nearby_points,1],
           c='orange', s =20, zorder=2)

axins.set_xlim(zoom_min[0], zoom_max[0])
axins.set_ylim(zoom_min[1], zoom_max[1])
axins.xaxis.set_visible('False')
axins.yaxis.set_visible('False')
_ = mark_inset(ax, axins, loc1=1, loc2=4, fc="none", ec="0.5")

We only need one nearby point, so we'll pick the first one

In [ ]:
nearby_point = nearby_points[0]
print(f'Nearby point index: {nearby_point}')
print(f'Nearby point value:\n{mesh.points[nearby_point,:]}')

## Comparing the points

Now we have a `starting_point` and `nearby_point`, we can compare them 'by hand' before we do a LEC analysis. Again, this is just as a tool to show how the various parts of the `gLEC` tool work.

### Elevation
The elevation data is stored on the mesh `point_data`. We can show the values:

In [ ]:
for point in (starting_point, nearby_point):
    print(f"Elevation of {point = } is {mesh.point_data['Z'][point]} m")

### Distance
The distance between the points can be calculated using `gLEC`'s `dist_func`. It uses a euclidean distance function.

In [ ]:
dist = lec_calculator.dist_func(starting_point, nearby_point)

In [ ]:
print(f"Distance between {starting_point = } and {nearby_point = } is\n{dist} m")

### Travel cost and the concept of 'fuel'

The measure of LEC is to determine the connectivity between points based on their elevation change. Therefore, we need to evaluate the 'cost' of moving from point to another

However, since we are potentially looking at global or regional scale meshes, it does not necessarily make sense to compare the elevation of points that are very far apart - for example, it is not particularly meaningful to compare the elevation of a point in Sydney, Australia to a point in Perth, Australia (some ~4,000 km away).

To mitigate this, `gLEC` (by default) uses two methods:
1. `gLEC`'s travel cost function is the change in elevation between two points, plus a small percentage of the horizontal distance between them (0.4% by default),
2. `gLEC` uses the concept of 'fuel' - that is, each path to a point has a maximum amount of 'fuel' it can use. Between each point, fuel is used up by the cost of the jump. When a path runs out of fuel, it stops. The default is for a `gLEC` object to have 2000 units of fuel.

Together, this means that the LEC analysis of a region is constrained in distance by the amount of fuel provided to `gLEC`. The normalised distance (i.e., the distance a path could cover with no elevation change) is therefore calculated by:

$$\frac{\text{max fuel}}{\text{horizontal distance fraction}} = \text{normalised distance}$$

In the default case then:

$$\frac{2000}{0.004} = 500,000 m$$

Worded another way, the longest path from a point could possibly be is 500 km long.

#### Travel cost

We can use the travel cost function on our `starting_point` and `nearby_point` as such:

In [ ]:
cost = lec_calculator.travel_cost_func(starting_point, nearby_point)

In [ ]:
print(f"The travel cost between {starting_point = } and {nearby_point = } is\n{cost} units of fuel")

If we compare this cost to the real elevation change between the points:

In [ ]:
elevation_change = abs(mesh.point_data['Z'][starting_point] - mesh.point_data['Z'][nearby_point])
print(f"{elevation_change = }")

We can see that the fuel cost is higher, since the horizontal distance is contributing to the cost.

# Performing a LEC analysis on a single point

Now let's use `gLEC` to perform a LEC analysis on our starting point, and look at the data it gives us.

In [ ]:
came_from, cost_so_far, dist_so_far = lec_calculator.cost_search(starting_point)

Before we look at those data structures, let's first visualise the paths that `gLEC` found to calculate the LEC:

In [ ]:
# VISUALISATION CODE

# Find all the nodes that are at the edge of the tree
edge_nodes = []
for k in came_from.keys():             # For all the points we've visited,
    if k not in came_from.values():    # Find all the points that haven't been 'came_from'
        edge_nodes.append(k)
        
# For each edge node, follow the path back to the starting point, and keep track of the points and costs along the way
paths = []
costs = []
dists = []
for p in edge_nodes:
    point = p
    cost = 0
    new_points = []
    new_costs = []
    while point:
        new_points.append(mesh.points[point])  # note, the points are being pulled from the VTK, so we get all their info
        new_costs.append(cost_so_far[point])
        point = came_from[point]

    new_points = np.array(new_points)
    new_costs  = np.array(new_costs)
    paths.append(new_points)
    costs.append(new_costs)

fig, (zm, ax) = plt.subplots(1,2,figsize=(12,15))

# Plot the mesh at full scale
ax.triplot(triang, zorder=1)

# Plot the starting point at full scale
ax.scatter(mesh.points[starting_point,0],
           mesh.points[starting_point,1],
           c='red', s =10, zorder=6)

norm = plt.Normalize(0, 1000)
for p, c in zip(paths, costs):    
    ax.plot(p[:,0], p[:,1], c='k', zorder=4)
    ax.scatter(p[:,0], p[:,1], s =10, c=c, norm=norm, zorder=5)
    
ax.set_aspect('equal')


# Plot the mesh 
zm.triplot(triang, zorder=1)

# Plot the starting point
zm.scatter(mesh.points[starting_point,0],
           mesh.points[starting_point,1],
           c='red', s =40, zorder=6)

# Plot the nearby points
for p, c in zip(paths, costs):    
    zm.plot(p[:,0], p[:,1], c='k', linewidth=4, zorder=4)
    zm.scatter(p[:,0], p[:,1], c=c, s=30, norm=norm, zorder=5)
zm.set_aspect('equal')
    
# Choose the size and zoom of the inset
zoom_min = np.min(np.vstack(paths), axis=0)
zoom_max = np.max(np.vstack(paths), axis=0)
span = zoom_max - zoom_min
zoom_min -= span * 0.1
zoom_max += span * 0.1
zm.set_xlim(zoom_min[0], zoom_max[0])
_ = zm.set_ylim(zoom_min[1], zoom_max[1])


The above image shows all the paths outwards from our `starting_point` (shown in red) that `gLEC` calculated. Note that these are all lowest-cost paths - each one is the 'cheapest' way from the starting point to each coloured point.



## Calculating the LEC value for our `starting_point`

Measuring LEC is more ambigious on a triangular mesh, since: 

+ A point could potentially have many connected neighbours,
+ The mesh cells may change resolution,
+ There are no boundary conditions (e.g., on a global or regional scale)

In contrast, calculating the LEC on a regular grid provides a fixed set of neighbours (4 or 8) which are evenly spaced.

This complexity means we need to define new ways to compare the LEC between points.

`gLEC` takes a relatively naive approach, and simply adds up the total distance of each path into a single number. This means that points that have paths that are long (i.e., for a given amount of a fuel, the path was able to reach points further away) have higher LEC values, and points that have mostly short paths have lower LEC values.

Since the paths trace out an area expanding from the initial point, the values tend to go up as the square of `max_fuel`.

`gLEC` has a function: `get_total_distance_for_all_paths_to_point` for doing this calculation. It runs the same `cost_search` function we used earlier, and then adds up the path distance at the end of each path:

In [ ]:
# Get a list of all the points we visted
all_visted_points = came_from.keys()

# We want a list of fully defined triangles - that is, where we visited all 3
# vertexes of the triangle.
neightris = []
for p in all_visted_points:
    # For our current point, find all the triangles it is part of
    neightris.extend(mesh.cells_dict['triangle'][np.where(mesh.cells_dict['triangle']==p)[0]])
neightris = np.unique(np.array(neightris), axis=0)

# For each triangle the point is in, see if the other vertexs were visited in all_visited_points
good_tris = []
for tri in neightris:
    if all(vertex in all_visted_points for vertex in tri):
        # If all points in the tri have been visited, then it's a 'good tri'
        good_tris.append(tri)
good_tris = np.array(good_tris)

def PolyArea(x,y):
    # From https://stackoverflow.com/a/30408825
    return 0.5*np.abs(np.dot(x,np.roll(y,1))-np.dot(y,np.roll(x,1)))

# For each good triangle, calculate the area, and add it to the total
area = 0.
for t in good_tris:
    points = mesh.points[t,:]
    area += PolyArea(points[:,0], points[:,1])

In [ ]:
print(area/1e6)

In [ ]:
# VISUALISATION CODE
triang = mtri.Triangulation(mesh.points[:,0], mesh.points[:,1], good_tris)
fig, ax = plt.subplots()
ax.triplot(triang, zorder=1)
ax.set_aspect('equal')

In [ ]:
lec_value = lec_calculator.get_total_distance_for_all_paths_to_point(starting_point)

In [ ]:
print(f"{lec_value = }")

Great, we now have a value for LEC at the `starting_point` we chose!

# Going forward

Having the LEC value for a single point is only useful within a context of the LEC of other points.

To do this, we need to calculate the LEC value for all the points in our mesh.

We will do this in the next notebook.